In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [21]:
clist = pd.read_csv("solar.csv")
print(clist.head())

                                  Company Name  \
0                                SunTrac Solar   
1                                    Capital E   
2  US Solar Energy Industries Association SEIA   
3                              FRONIUS USA LLC   
4           Array Technologies Inc aka Wattsun   

                                 Company Name# Address  \
0                                SunTrac Solar     NaN   
1                                    Capital E     NaN   
2  US Solar Energy Industries Association SEIA     NaN   
3                              FRONIUS USA LLC     NaN   
4           Array Technologies Inc aka Wattsun     NaN   

                                    Place       Zip          Sector  \
0                       Golden,, Colorado     80403           Solar   
1              Washington, Washington, DC  DC 20003  Services,Solar   
2              Washington, Washington, DC     20005           Solar   
3  Brighton, Michigan 48116 USA, Michigan       NaN           Sola

In [22]:
cname = clist["Company Name"]
cname_list = []
for c in cname:
    string = c.replace(' ','_')
    url_st = ''.join(["http://en.openei.org/wiki/", string])
    cname_list.append(url_st)
test_list = (cname_list[0:20])
print(test_list)

['http://en.openei.org/wiki/SunTrac_Solar', 'http://en.openei.org/wiki/Capital_E', 'http://en.openei.org/wiki/US_Solar_Energy_Industries_Association_SEIA', 'http://en.openei.org/wiki/FRONIUS_USA_LLC', 'http://en.openei.org/wiki/Array_Technologies_Inc_aka_Wattsun', 'http://en.openei.org/wiki/Solar_International_Management_Inc', 'http://en.openei.org/wiki/Solarponics', 'http://en.openei.org/wiki/A1_Sun,_Inc.', 'http://en.openei.org/wiki/PetersenDean_Roofing_and_Solar', 'http://en.openei.org/wiki/Alten_Products_Company', 'http://en.openei.org/wiki/AMECO_Solar', 'http://en.openei.org/wiki/Ameco_Solar', 'http://en.openei.org/wiki/Run_on_Sun', 'http://en.openei.org/wiki/3Degrees', 'http://en.openei.org/wiki/JouleBox_Power', 'http://en.openei.org/wiki/Solaris_Technology_Industry,_Inc.', 'http://en.openei.org/wiki/Wave_Energy_-_Efficiency_Solutions', 'http://en.openei.org/wiki/Alternate_Energy_LLC', 'http://en.openei.org/wiki/ALDACOR_INC', 'http://en.openei.org/wiki/Inovus']


In [23]:
scrape_list = []
scrape_data = []
structure_df = [{'Name': 'NA',  'Website2': 'NA', 'Phone Number': 'NA'}]
df = pd.DataFrame(structure_df)
for url in cname_list:
    test_df = pd.DataFrame(structure_df)
    temp_data = []
    temp_list = []
    response = requests.get(url)
    content = response.content
    parser = BeautifulSoup(content, 'html.parser')
    body = parser.find_all("body")
    th = body[0].find_all("th")
    td = body[0].find_all("td")
    for p in th:
        temp_list.append(p.text)
    for p in td:
        temp_data.append(p.text)
    if temp_data[0] != " ":
        test_df["Name"].loc[0] = temp_data[0]
    if temp_data[0] == " ":
        test_df["Name"].loc[0] = temp_data[1]
    if "Website:" in temp_list:
        for row in temp_data:
            if re.search("www|http", row):
                test_df["Website2"].loc[0] = row
    if "Phone Number:" in temp_list:
        for row in temp_data:
            if re.search("[0-9][0-9][0-9]\-[0-9][0-9][0-9][0-9]$|[0-9][0-9][0-9]\.[0-9][0-9][0-9][0-9]$|[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]", row):
                test_df["Phone Number"].loc[0] = row
    df = pd.concat([df, test_df])

In [24]:
clist["Name"] = clist["Company Name"]

In [25]:
clist_name_index = clist.set_index("Name")

In [32]:
df_name_index = df.set_index("Name")

In [31]:
print(clist.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822 entries, 0 to 821
Data columns (total 14 columns):
Company Name     822 non-null object
Company Name#    822 non-null object
Address          320 non-null object
Place            796 non-null object
Zip              709 non-null object
Sector           822 non-null object
Product          745 non-null object
Website          339 non-null object
Region           180 non-null object
Place.1          796 non-null object
1                725 non-null object
2                33 non-null object
Unnamed: 12      1 non-null object
Name             822 non-null object
dtypes: object(14)
memory usage: 90.0+ KB
None


In [26]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 823 entries, 0 to 0
Data columns (total 3 columns):
Name            823 non-null object
Phone Number    823 non-null object
Website2        823 non-null object
dtypes: object(3)
memory usage: 25.7+ KB
None


In [28]:
print(clist_name_index.info())

<class 'pandas.core.frame.DataFrame'>
Index: 822 entries, SunTrac Solar to Tahoe Water Systems
Data columns (total 13 columns):
Company Name     822 non-null object
Company Name#    822 non-null object
Address          320 non-null object
Place            796 non-null object
Zip              709 non-null object
Sector           822 non-null object
Product          745 non-null object
Website          339 non-null object
Region           180 non-null object
Place.1          796 non-null object
1                725 non-null object
2                33 non-null object
Unnamed: 12      1 non-null object
dtypes: object(13)
memory usage: 89.9+ KB
None


In [33]:
clean_df_left = df_name_index.join(clist_name_index, how='left')

In [35]:
print(len(clean_df_left))
print(clean_df_left.head())

823
                          Phone Number                           Website2  \
Name                                                                        
1-Solar Inc                         NA                                 NA   
1366 Technologies                   NA                 www.1366tech.com/    
1st Light Energy, Inc.    209-824-5500                                 NA   
200px                               NA  http://facebook.com/jouleboxpower   
200px                   1-877-837-6527               www.intelisolar.com    

                                  Company Name           Company Name#  \
Name                                                                     
1-Solar Inc                                NaN                     NaN   
1366 Technologies            1366 Technologies       1366 Technologies   
1st Light Energy, Inc.  1st Light Energy, Inc.  1st Light Energy, Inc.   
200px                                      NaN                     NaN   
200px       

In [ ]:
clean_df_left.to_csv("cleansolar3.csv")